In [1]:
from general import *
from exp2 import *

random_state=42
np.random.seed(random_state)

In [184]:
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import SplineTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from scipy.stats import norm

## Experiment 

In [185]:
states=['ca','il','mo','tx']
labels = ['California','Illinois','Missouri','Texas']
alpha=.05
loss='mae'
colors=['#EB9846', '#12B8EB', '#773BEB', '#2F58EB', '#808080']

In [186]:
data = pd.read_csv('data/car-insurance-public/data/mo-per-zip.csv')
data.head()

,zipcode,minority,companies_name,name,bi_policy_premium,pd_policy_premium,state_risk,white_non_hisp_pct,risk_difference,combined_premium
0,63005,False,21st Century Centennial Ins Co,FARMERS INS GRP,321,211.0,216.029693,87.9,315.970307,532.0
1,63010,False,21st Century Centennial Ins Co,FARMERS INS GRP,342,192.0,251.359806,93.9,282.640194,534.0
2,63011,False,21st Century Centennial Ins Co,FARMERS INS GRP,321,211.0,225.421778,88.9,306.578222,532.0
3,63012,False,21st Century Centennial Ins Co,FARMERS INS GRP,342,192.0,210.286312,97.2,323.713688,534.0
4,63013,False,21st Century Centennial Ins Co,FARMERS INS GRP,317,141.0,158.592140,100.0,299.407860,458.0


In [195]:
pvals=[]

for s in states:
    data = pd.read_csv('data/car-insurance-public/data/' + s + '-per-zip.csv')

    Z = np.array(data.state_risk).reshape((-1,1))
    Y = np.array(data.combined_premium).reshape((-1,1))
    X = (1*np.array(data.minority)).reshape((-1,1))
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=.5, random_state=random_state)
    Z_train, Z_test, _, _ = train_test_split(Z, Y, test_size=.5, random_state=random_state)
    
    
    ####
    reg1 = LinearRegression().fit(Z_train, Y_train.squeeze())
    
    pipeline = Pipeline([
    ('spline', SplineTransformer()),
    ('lr', LinearRegression())
    ])

    params = {
        'spline__degree': [1, 2, 3, 4, 5],
        'spline__n_knots': [2,3,4,5,6],
        'spline__knots': ['uniform', 'quantile'],
        'spline__extrapolation': ['linear', 'continue']
    }

    grid_search = GridSearchCV(pipeline, params, cv=10, n_jobs=-1)
    grid_search = grid_search.fit(Z_train, Y_train)
    
    spline = SplineTransformer(degree=grid_search.best_params_['spline__degree'], 
                           n_knots=grid_search.best_params_['spline__n_knots'], 
                           knots=grid_search.best_params_['spline__knots'])

    reg2 = LinearRegression().fit(spline.fit_transform(Z_train), Y_train.squeeze())
    
    errors1 = (reg1.predict(Z_test) - Y_test.squeeze())**2
    errors2 = (reg2.predict(spline.fit_transform(Z_test)) - Y_test.squeeze())**2
    d = errors1-errors2

    n = errors1.shape[0]
    T = abs(n**.5*d.mean()/d.std())
    print(1-norm.cdf(T))

4.136571540858114e-08
7.360778653264788e-14
0.0033236007749919416
0.1817614805075538
